# 9. Clustering and k-means algorithm

## Simple clustering example

So far we have been looking only at supervised learning problems, where the training data consists of previously labeled samples. For these samples, the correct value of the target variable is known from the beginning. However, it is also possible to do meaningful machine learning even when such labels are not available. A very common example of such **unsupervised learning** has to do with **clustering**: in clustering problems, there is a training set containing samples with a number of variables, but no particular "target variable" to make predictions about. Instead, one asks the following question: is it possible to divide the samples in subgroups in such a way that the samples in one of them are (in some sense to be more precisely defined below) "close" to each other, but "distant" from those in other groups? Such questions arise very naturally e.g. in the context of market segmentation or targeted advertising. 

As a simple example, let us use once more the familiar iris dataset: 

In [3]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

However, this time we shall ignore (for now, at least) the target values, and focus our attention to the values of the four numerical variables for each sample (contained in the array `X`). Let us also assume that we expect there to be three distinct species of flowers in the dataset, and we would like to find them out automatically. That is, we ask whether the samples in the dataset naturally fall into three subgroups according to their variable values. This is an example of a clustering problem.

One of the most commonly used algorithm for finding the desired subdivision is the **k-means algorithm**. Implementing k-means clustering in Scikit-learn is straightforward: essentially, only the desired number of clusters (the parameter `n_clusters` below) needs to be specified.   

In [16]:
from sklearn.cluster import KMeans

model = KMeans(init='random', n_clusters=3, random_state=42)
model.fit(X)

KMeans(init='random', n_clusters=3, random_state=42)

After the model has been trained, we can find the cluster labels of the training samples as follows:

In [27]:
labels = model.labels_
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1])

After clustering, each of the 150 training samples are assigned to one of three possible clusters (labelled either 0, 1, or 2). Recall that the training set is arranged so that the first 50 samples are of type "setosa", next 50 "versicolor", and the last 50 "virginica". Let us compare the clustering results to that of species values: 

In [29]:
print('Class setosa:', labels[:50])
print('Class versicolor:', labels[50:100])
print('Class virginica:', labels[100:])

Class setosa: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
Class versicolor: [1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
Class virginica: [2 1 2 2 2 2 1 2 2 2 2 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2
 2 1 2 2 2 1 2 2 2 1 2 2 1]


We find that all the samples of class setosa (and those only) really ended up in the same cluster. In the other two classes there is some mixing, but this simple clustering exercise was able to place the different flower samples in three classes according to their identity with quite reasonable success. Next, we take a closer look at how the k-means clustering actually works.

## K-means algorithm

Consider a training set with N samples, each characterized by their M separate variable values $x_{1}^{(i)}, x_{2}^{(i)}, ..., x_{M}^{(i)}$, $i = 1,...,N$; in our iris example, N = 150 and M = 4. K-means clustering involves a single hyperparameter k: the number of clusters the training set samples are subdivided into. The algorithm proceeds to find this cluster division as follows:

1) First, k **means/cluster centers/centroids** are selected. Each centroid is a point in the same M-dimensional space as the training data samples, with coordinates $m_{1}^{(j)}, m_{2}^{(j)}, ..., m_{M}^{(j)}$, $j = 1,...,k$. Often the cluster centroids are initialized randomly: their coordinate values might e.g. be taken to coincide with those of k randomly selected samples from the training set.

2) Next, each sample is **assigned** to the cluster whose centroid is nearest to it. For this purpose, the Euclidean distance (see module 3) is usually used. For each sample denoted by index i, the distance to each of the k clusters is computed with the expression 
$$
d^{(i,j)} = \sqrt{(x_{1}^{(i)}-m_{1}^{(j)})^{2}+...+(x_{M}^{(i)}-m_{M}^{(j)})^{2}}, \hspace{5mm} j=1,...,k 
$$ 
and each sample is assigned to the cluster whose centroid has the shortest distance to it.

3) After the assignment step has been completed, the cluster centroids are **updated**. The most common strategy for updating the centroids is to take the *averages* of the coordinate values of the individual samples assigned to that particular cluster. Consider, e.g. that $N_{j}$ samples have been assigned to a cluster with index $j$; in that case, the updated value for the first coordinate $m_{1}^{(j)}$ for that centroid is calculated as (the sum is taken over the samples assigned to cluster j)
$$
m_{1}^{(j)} = \frac{1}{N_{j}}\sum_{i=1}^{N_j} x_{1}^{(i)},
$$
and similarly for all the other M-1 coordinates.

4) The two previous steps (assignment + update) are repeated until the algorithm has converged, and the cluster assignments no longer change.

The objective function for the k-means clustering problem is the sum of squared distances for each sample to its assigned cluster center. This problem, however, is computationally hard (NP-hard), and the k-means algorithm is not guaranteed to solve it: the converged solution is not necessarily the optimal one. Furthermore, the final solution may also depend on the initialization. Sometimes it may be necessary, therefore, to run the algorithm several times with different random initializations.

After the cluster assignments have been computed, and the converged solution has been found, the variable/coordinate values of the final cluster centers can be accessed as follows:  

In [31]:
model.cluster_centers_

array([[5.006     , 3.428     , 1.462     , 0.246     ],
       [5.9016129 , 2.7483871 , 4.39354839, 1.43387097],
       [6.85      , 3.07368421, 5.74210526, 2.07105263]])